In [ ]:
## Stock price prediction using BERT and GAN, 2017
# LSTM -> Bi-LSTM

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from keras import optimizers
from keras import backend as K

import random
import train_test as tt
import os

2023-04-14 01:48:31.842977: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-14 01:48:31.943030: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-14 01:48:31.946082: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-14 01:48:31.946098: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [8]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)

seed_everything(42)

In [9]:
################################### 논문 재현(변수 다르고, APPLE 사용) #######################

In [10]:
dir = "/home/whfhrs3260/csv_data/price_data_score_10years.csv"
stock = "AAPL"
variable = ["before_close","Score"]
window_size = 3
start_date = "2010-07-01"
end_date = "2020-07-31"

scaler_x, scaler_y,x, y, x_t, y_t = tt.train_test_result(dir, stock, variable, window_size, start_date, end_date)

bi_lstm_model = Sequential()
bi_lstm_model.add(Bidirectional(LSTM(128, input_shape = (x.shape[1],x.shape[2]))))
bi_lstm_model.add(Dense(64))
bi_lstm_model.add(Dense(1,activation="tanh")) 
adam = optimizers.Adam(learning_rate=0.0012)
bi_lstm_model.compile(optimizer=adam, loss = "mse")
history = bi_lstm_model.fit(x, y, epochs=150, batch_size=64, validation_split=0.2)
    
bi_lstm_pred = bi_lstm_model.evaluate(x_t, y_t)

[*********************100%***********************]  1 of 1 completed


/home/whfhrs3260/Python/10year_data/review/train_test.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set_news_data_date.price_date[i]=standard  + datetime.timedelta(days=1)


[*********************100%***********************]  1 of 1 completed
Epoch 1/150


/home/whfhrs3260/Python/10year_data/review/train_test.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data.before_close[i+1] = price_data.Close[i]


23/23 [==============================] - 3s 32ms/step - loss: 0.0041 - val_loss: 0.0012
Epoch 2/150
23/23 [==============================] - 0s 10ms/step - loss: 2.1951e-04 - val_loss: 4.5164e-05
Epoch 3/150
23/23 [==============================] - 0s 11ms/step - loss: 4.8686e-05 - val_loss: 5.5531e-05
Epoch 4/150
23/23 [==============================] - 0s 11ms/step - loss: 2.1625e-05 - val_loss: 2.5378e-05
Epoch 5/150
23/23 [==============================] - 0s 11ms/step - loss: 1.9758e-05 - val_loss: 3.4613e-05
Epoch 6/150
23/23 [==============================] - 0s 11ms/step - loss: 2.0268e-05 - val_loss: 3.3834e-05
Epoch 7/150
23/23 [==============================] - 0s 11ms/step - loss: 1.9691e-05 - val_loss: 3.1085e-05
Epoch 8/150
23/23 [==============================] - 0s 11ms/step - loss: 2.0060e-05 - val_loss: 3.7102e-05
Epoch 9/150
23/23 [==============================] - 0s 11ms/step - loss: 2.0753e-05 - val_loss: 2.8183e-05
Epoch 10/150
23/23 [============================

In [11]:
bi_lstm_pred = bi_lstm_model.predict(x_t)
bi_lstm_pred = scaler_y.inverse_transform(bi_lstm_pred)
y_t = scaler_y.inverse_transform(y_t)
root_mean_squared_error(y_t, bi_lstm_pred), mean_squared_error(y_t, bi_lstm_pred), mean_absolute_error(y_t, bi_lstm_pred), MAPE(y_t, bi_lstm_pred)

24/24 [==============================] - 1s 3ms/step


(<tf.Tensor: shape=(), dtype=float64, numpy=4.769939369398908>,
 22.752321587741648,
 3.1731552927117597,
 4.936982723174324)

In [73]:
#################################### 2 variable(news, close 사용) #####################################

In [18]:
dir = "/home/whfhrs3260/csv_data/price_data_score_10years.csv"
stock = "^DJI"
variable = ["before_close","Score"]
window_size = 3
start_date = "2012-01-01"
end_date = "2022-04-30"

scaler_x, scaler_y,x, y, x_t, y_t = tt.train_test_result(dir, stock, variable, window_size, start_date, end_date)

bi_lstm_model = Sequential()
bi_lstm_model.add(Bidirectional(LSTM(128, input_shape = (x.shape[1],x.shape[2]))))
bi_lstm_model.add(Dense(64))
bi_lstm_model.add(Dense(1,activation="tanh")) 
adam = optimizers.Adam(learning_rate=0.0012)
bi_lstm_model.compile(optimizer=adam, loss = "mse")
history = bi_lstm_model.fit(x, y, epochs=150, batch_size=64, validation_split=0.2)
    
bi_lstm_pred = bi_lstm_model.evaluate(x_t, y_t)

[*********************100%***********************]  1 of 1 completed


/home/whfhrs3260/Python/10year_data/review/train_test.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set_news_data_date.price_date[i]=standard  + datetime.timedelta(days=1)


[*********************100%***********************]  1 of 1 completed
Epoch 1/150


/home/whfhrs3260/Python/10year_data/review/train_test.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data.before_close[i+1] = price_data.Close[i]


23/23 [==============================] - 3s 33ms/step - loss: 0.0102 - val_loss: 0.0021
Epoch 2/150
23/23 [==============================] - 0s 10ms/step - loss: 5.2329e-04 - val_loss: 0.0012
Epoch 3/150
23/23 [==============================] - 0s 11ms/step - loss: 1.4892e-04 - val_loss: 0.0018
Epoch 4/150
23/23 [==============================] - 0s 11ms/step - loss: 8.3841e-05 - val_loss: 0.0013
Epoch 5/150
23/23 [==============================] - 0s 12ms/step - loss: 6.5307e-05 - val_loss: 0.0011
Epoch 6/150
23/23 [==============================] - 0s 12ms/step - loss: 5.2056e-05 - val_loss: 8.9385e-04
Epoch 7/150
23/23 [==============================] - 0s 11ms/step - loss: 4.6215e-05 - val_loss: 7.4312e-04
Epoch 8/150
23/23 [==============================] - 0s 11ms/step - loss: 4.4231e-05 - val_loss: 7.1460e-04
Epoch 9/150
23/23 [==============================] - 0s 11ms/step - loss: 4.4728e-05 - val_loss: 7.4582e-04
Epoch 10/150
23/23 [==============================] - 0s 11ms/st

In [19]:
bi_lstm_pred = bi_lstm_model.predict(x_t)
bi_lstm_pred = scaler_y.inverse_transform(bi_lstm_pred)
y_t = scaler_y.inverse_transform(y_t)
root_mean_squared_error(y_t, bi_lstm_pred), mean_squared_error(y_t, bi_lstm_pred), mean_absolute_error(y_t, bi_lstm_pred), MAPE(y_t, bi_lstm_pred)

25/25 [==============================] - 1s 3ms/step


(<tf.Tensor: shape=(), dtype=float64, numpy=1904.3791837918943>,
 3626660.075659882,
 1481.821582784383,
 4.553761019666613)

In [26]:
dir_model = "/home/whfhrs3260/model/"
#bi_lstm_model.save(dir_model + 'bi_lstm_128_64_window30_10years.h5')

In [ ]:
# model load

In [30]:
from keras.models import load_model

In [32]:
dir_model = "/home/whfhrs3260/model/"
dir = "/home/whfhrs3260/csv_data/price_data_score_10years.csv"
stock = "^DJI"
variable = ["before_close","Score"]
window_size = 30
start_date = "2012-01-01"
end_date = "2022-04-30"

scaler_x, scaler_y,x, y, x_t, y_t = tt.train_test_result(dir, stock, variable, window_size, start_date, end_date)

bi_lstm_model = load_model(dir_model + 'bi_lstm_128_64_window30_10years.h5', custom_objects ={"root_mean_squared_error" : root_mean_squared_error})
bi_lstm_pred = bi_lstm_model.predict(x_t)

[*********************100%***********************]  1 of 1 completed


/home/whfhrs3260/Python/10year_data/review/train_test.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set_news_data_date.price_date[i]=standard  + datetime.timedelta(days=1)


[*********************100%***********************]  1 of 1 completed


/home/whfhrs3260/Python/10year_data/review/train_test.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data.before_close[i+1] = price_data.Close[i]


24/24 [==============================] - 1s 22ms/step


In [33]:
bi_lstm_pred = scaler_y.inverse_transform(bi_lstm_pred)
y_t = scaler_y.inverse_transform(y_t)
root_mean_squared_error(y_t, bi_lstm_pred), mean_squared_error(y_t, bi_lstm_pred), mean_absolute_error(y_t, bi_lstm_pred), MAPE(y_t, bi_lstm_pred)

(<tf.Tensor: shape=(), dtype=float64, numpy=742.1500555456893>,
 550786.7049464696,
 561.2534303220707,
 1.8152156454865287)